In [228]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [229]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [230]:
# Import Dataset
df = pd.read_csv('./la_text_data.csv', quoting = 3).dropna().reset_index(drop=True)

In [231]:
word_replace_dic = {
        ' vic ':' victim ',
        ' vict ':' victim ',
        ' sus ': ' suspect ',
        ' s1 ': ' suspect ',
        ' v1 ': ' victim ',
        ' loc ': ' location ',
        ' veh ':' vehicle ',
        ' prop ': ' property ',
        ' unk ': ' unknown ',
        ' v ' : ' victim ',
        ' s ': ' suspect ',
        ' ss ': ' suspect ',
        ' susp ': ' suspect ',
        ' remvd ': ' removed ',
        ' victs ': ' victims ',
        ' susps ': ' suspects ',
        ' stillinside ': ' still inside ',
        ' donttell ': ' dont tell ',
        ' veerbal ': ' verbal ',
        ' thransaction ': ' transaction ',
        ' usedcredit ': ' used credit ',
        ' beerbottle ' : ' beer bottle ',
        ' neg ': ' negative ',
        ' cointinuously ': ' continuously ',
        ' lemmon ': ' lemon ',
        ' att ': ' attack '
            }
df['content'] = " " + df['NARRATIVE'].str.lower()+ " "

In [232]:
for key, value in word_replace_dic.items():
    df['content'] = df['content'].str.replace(key, value, 10)

In [11]:
df['NARRATIVE'][2673]

'SUSPS REPEATEDLY CALL VICT VICT AHS ASKED SUSPS TO STOP CALLING HIM WITH NEG RESULTS SUSPS PHONE CALLS ARE ANNOYING TO VICT'

In [12]:
df['content'][2673]

' suspects repeatedly call victim victim ahs asked suspects to stop calling him with negative results suspects phone calls are annoying to victim '

In [233]:
for key, value in word_replace_dic.items():
    df['content'] = df['content'].str.replace(key, value, 10)

In [234]:
df[df['content'].str.contains(" vict ")]['content']

Series([], Name: content, dtype: object)

In [235]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])

[' suspect continiously calls and text victim harrassing messages victim has repeatedly advised suspect to stop making contact ']


### Tokenize words and Clean-up text

In [236]:
#I have set deacc=True to remove the punctuations.
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['suspect', 'continiously', 'calls', 'and', 'text', 'victim', 'harrassing', 'messages', 'victim', 'has', 'repeatedly', 'advised', 'suspect', 'to', 'stop', 'making', 'contact']]


In [237]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['suspect', 'continiously', 'calls', 'and', 'text', 'victim', 'harrassing', 'messages', 'victim', 'has', 'repeatedly', 'advised', 'suspect', 'to', 'stop', 'making', 'contact']


In [238]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [239]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['suspect', 'continiously', 'call', 'text', 'victim', 'harrasse', 'message', 'victim', 'repeatedly', 'advise', 'suspect', 'stop', 'make', 'contact']]


### Create the Dictionary and Corpus needed for Topic Modeling

In [240]:
#The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 2)]]


In [21]:
id2word[0]

'advise'

In [22]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('advise', 1),
  ('call', 1),
  ('contact', 1),
  ('continiously', 1),
  ('harrasse', 1),
  ('make', 1),
  ('message', 1),
  ('repeatedly', 1),
  ('stop', 1),
  ('suspect', 2),
  ('text', 1),
  ('victim', 2)]]

In [23]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, #We have 12 crime type in our real data
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [24]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.124*"go" + 0.123*"state" + 0.061*"get" + 0.049*"kill" + 0.038*"be" + '
  '0.035*"discover" + 0.035*"miss" + 0.033*"come" + 0.027*"wallet" + '
  '0.026*"wall"'),
 (1,
  '0.394*"rear" + 0.141*"attack" + 0.113*"car" + 0.062*"access" + 0.047*"seat" '
  '+ 0.046*"shatter" + 0.046*"slide" + 0.034*"trunk" + 0.011*"various" + '
  '0.008*"bumper"'),
 (2,
  '0.000*"borg" + 0.000*"feudng" + 0.000*"anddont" + 0.000*"responsed" + '
  '0.000*"stabbee" + 0.000*"boxconceale" + 0.000*"pocketsthen" + 0.000*"osver" '
  '+ 0.000*"exsusp" + 0.000*"andvictim"'),
 (3,
  '0.049*"strike" + 0.048*"time" + 0.042*"push" + 0.041*"grab" + '
  '0.037*"approach" + 0.037*"business" + 0.034*"throw" + 0.031*"purse" + '
  '0.030*"head" + 0.030*"attempt"'),
 (4,
  '0.139*"money" + 0.121*"walk" + 0.070*"threaten" + 0.068*"tell" + '
  '0.041*"run" + 0.034*"upset" + 0.032*"steal" + 0.028*"kill" + 0.027*"fail" + '
  '0.024*"metal"'),
 (5,
  '0.139*"pay" + 0.070*"order" + 0.065*"fear" + 0.056*"give" + 0.048*"back" + 

In [25]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -5.946351205832915

Coherence Score:  0.29532926394004894


In [26]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\Samira\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.218896 -0.242247       1        1  67.838654
6     -0.143977 -0.158031       2        1   7.367177
3     -0.136793  0.059468       3        1   7.349777
0     -0.069357  0.282687       4        1   3.158078
5     -0.069319 -0.085362       5        1   2.941467
11    -0.042219  0.017300       6        1   2.853594
4     -0.039729  0.254155       7        1   2.627007
9     -0.043882  0.016584       8        1   2.346380
8     -0.015648  0.002402       9        1   2.121819
1      0.076807 -0.033831      10        1   1.384861
7      0.351507 -0.056563      11        1   0.005591
2      0.351507 -0.056563      12        1   0.005590, topic_info=      Category          Freq         Term         Total  loglift  logprob
9      Default  1.227810e+06      suspect  1.227810e+06  30.0000  30.0000
11     Default  1.196522e+06       victim  1.196522e+06  29.0000  29.0000
27     Default  6.004740e+05      unknown  6.004740e+05  28.0000  28.0000
203    Default  5.185600e+04         rear  5.185600e+04  27.0000  27.0000
47     Default  1.057450e+05          use  1.057450e+05  26.0000  26.0000
72     Default  4.370600e+04        entry  4.370600e+04  25.0000  25.0000
64     Default  3.860800e+04         face  3.860800e+04  24.0000  24.0000
19     Default  3.732900e+04        punch  3.732900e+04  23.0000  23.0000
73     Default  3.978000e+04         gain  3.978000e+04  22.0000  22.0000
148    Default  3.712700e+04        cause  3.712700e+04  21.0000  21.0000
37     Default  3.880100e+04          pay  3.880100e+04  20.0000  20.0000
173    Default  3.734700e+04           go  3.734700e+04  19.0000  19.0000
109    Default  3.686600e+04        state  3.686600e+04  18.0000  18.0000
123    Default  3.393900e+04       verbal  3.393900e+04  17.0000  17.0000
180    Default  3.469800e+04        money  3.469800e+04  16.0000  16.0000
132    Default  3.448700e+04        phone  3.448700e+04  15.0000  15.0000
15     Default  2.972380e+05         flee  2.972380e+05  14.0000  14.0000
84     Default  3.022400e+04       become  3.022400e+04  13.0000  13.0000
46     Default  2.866450e+05     property  2.866450e+05  12.0000  12.0000
325    Default  3.013100e+04         walk  3.013100e+04  11.0000  11.0000
28     Default  2.644030e+05      vehicle  2.644030e+05  10.0000  10.0000
40     Default  2.624400e+04          cut  2.624400e+04   9.0000   9.0000
314    Default  3.524900e+04     argument  3.524900e+04   8.0000   8.0000
34     Default  3.230100e+04         exit  3.230100e+04   7.0000   7.0000
215    Default  2.400400e+04       damage  2.400400e+04   6.0000   6.0000
68     Default  3.425500e+04       strike  3.425500e+04   5.0000   5.0000
20     Default  3.360500e+04         time  3.360500e+04   4.0000   4.0000
120    Default  2.321000e+04      dispute  2.321000e+04   3.0000   3.0000
202    Default  1.852600e+04       attack  1.852600e+04   2.0000   2.0000
17     Default  1.960190e+05     location  1.960190e+05   1.0000   1.0000
...        ...           ...          ...           ...      ...      ...
72591  Topic12  7.318685e-03          mem  8.754261e-01   5.0076 -11.1926
24199  Topic12  7.318685e-03      butkept  8.720528e-01   5.0115 -11.1926
24193  Topic12  7.318685e-03         hunk  8.720528e-01   5.0115 -11.1926
24194  Topic12  7.318685e-03         fard  8.720528e-01   5.0115 -11.1926
24195  Topic12  7.318685e-03        ofbee  8.720528e-01   5.0115 -11.1926
24196  Topic12  7.318685e-03        sacke  8.720528e-01   5.0115 -11.1926
24197  Topic12  7.318685e-03     passerng  8.720528e-01   5.0115 -11.1926
24198  Topic12  7.318685e-03        nokia  8.720528e-01   5.0115 -11.1926
24200  Topic12  7.318685e-03       suspwa  8.720528e-01   5.0115 -11.1926
24191  Topic12  7.318685e-03         recp  8.720528e-01   5.0115 -11.1926
24201  Topic12  7.318685e-03          dbt  8.720528e-01   5.0115 -11.1926
24202  Topic12  7.3

In [241]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
#mallet_path = 'F:\\mallet-2.0.8\\bin\\mallet'
mallet_path = "F:\\mallet-2.0.8\\bin\\mallet"
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=12, id2word=id2word)

In [242]:
# Show Topics
pprint(ldamallet.show_topics(num_topics=12, formatted=False))

[(0,
  [('suspect', 0.19432459298264468),
   ('victim', 0.19097576701950458),
   ('money', 0.04425742840255763),
   ('approach', 0.036755144806834665),
   ('foot', 0.02629090322927301),
   ('demand', 0.023585513943366825),
   ('give', 0.022916554725699857),
   ('walk', 0.021108484229756596),
   ('gun', 0.019397130729138682),
   ('approached', 0.019308473483423783)]),
 (1,
  [('suspect', 0.4428832772259859),
   ('victim', 0.23082952555181507),
   ('flee', 0.11880974457930628),
   ('location', 0.0980472415511802),
   ('unknown', 0.03830335836584218),
   ('property', 0.03380920160497643),
   ('scene', 0.0065825195679595026),
   ('left', 0.004336833429398022),
   ('removed', 0.0011430305764159796),
   ('pro', 0.0011332848833162088)]),
 (2,
  [('victim', 0.2325558471313823),
   ('suspect', 0.07606266068122178),
   ('face', 0.04238254452451679),
   ('punch', 0.04176579504735517),
   ('time', 0.03999547095416339),
   ('strike', 0.03944932130916066),
   ('push', 0.029482756324013266),
   ('hit

In [37]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)
# Compute Perplexity
#print('\nPerplexity: ', ldamallet.log_perplexity(corpus))  # a measure of how good the model is. lower the better.



Coherence Score:  0.5327888090297348


In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

If the coherence score seems to keep increasing, it may make better sense to pick the model that gave the highest CV before flattening out. This is exactly the case here.

So for further steps I will choose the model with 12 topics itself.


In [ ]:
# Select the model and print the topics
optimal_model = model_list[2]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [40]:
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, id2word)
vis

C:\Users\Samira\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
C:\Users\Samira\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
C:\Users\Samira\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
C:\Users\Samira\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
10    -0.338074  0.113381       1        1  8.464208
0     -0.309180 -0.022757       2        1  8.415539
7     -0.310318 -0.041791       3        1  8.413331
8      0.178444  0.157978       4        1  8.405783
5     -0.401640  0.039771       5        1  8.371080
9      0.198413  0.099023       6        1  8.353365
3      0.208773 -0.011500       7        1  8.338507
4      0.223935  0.002682       8        1  8.289084
6      0.189324 -0.035040       9        1  8.274765
1      0.168467 -0.056757      10        1  8.248566
11     0.042473 -0.384022      11        1  8.234389
2      0.149385  0.139032      12        1  8.191382, topic_info=    Category          Freq         Term         Total  loglift  logprob
27   Default  6.515400e+05      unknown  6.515400e+05  30.0000  30.0000
11   Default  1.202724e+06       victim  1.202724e+06  29.0000  29.0000
9    Default  1.242052e+06      suspect  1.242052e+06  28.0000  28.0000
28   Default  2.663540e+05      vehicle  2.663540e+05  27.0000  27.0000
46   Default  3.146770e+05     property  3.146770e+05  26.0000  26.0000
15   Default  2.997450e+05         flee  2.997450e+05  25.0000  25.0000
17   Default  2.021590e+05     location  2.021590e+05  24.0000  24.0000
38   Default  1.804530e+05       remove  1.804530e+05  23.0000  23.0000
75   Default  1.037410e+05       window  1.037410e+05  22.0000  22.0000
43   Default  1.285760e+05        enter  1.285760e+05  21.0000  21.0000
143  Default  8.528400e+04         door  8.528400e+04  20.0000  20.0000
114  Default  7.714600e+04         open  7.714600e+04  19.0000  19.0000
74   Default  6.056100e+04        smash  6.056100e+04  18.0000  18.0000
109  Default  4.369200e+04        state  4.369200e+04  17.0000  17.0000
22   Default  4.888500e+04          dir  4.888500e+04  16.0000  16.0000
99   Default  4.271300e+04         side  4.271300e+04  15.0000  15.0000
73   Default  4.261700e+04         gain  4.261700e+04  14.0000  14.0000
72   Default  4.159800e+04        entry  4.159800e+04  13.0000  13.0000
35   Default  4.002700e+04         item  4.002700e+04  12.0000  12.0000
168  Default  3.983600e+04         lock  3.983600e+04  11.0000  11.0000
369  Default  3.953300e+04        store  3.953300e+04  10.0000  10.0000
182  Default  5.379900e+04    direction  5.379900e+04   9.0000   9.0000
95   Default  5.111500e+04        leave  5.111500e+04   8.0000   8.0000
37   Default  3.797900e+04          pay  3.797900e+04   7.0000   7.0000
64   Default  3.300000e+04         face  3.300000e+04   6.0000   6.0000
19   Default  3.251900e+04        punch  3.251900e+04   5.0000   5.0000
203  Default  4.509000e+04         rear  4.509000e+04   4.0000   4.0000
123  Default  3.169600e+04       verbal  3.169600e+04   3.0000   3.0000
34   Default  3.093300e+04         exit  3.093300e+04   2.0000   2.0000
20   Default  3.114200e+04         time  3.114200e+04   1.0000   1.0000
..       ...           ...          ...           ...      ...      ...
433  Topic12  1.667370e+04     personal  1.667370e+04   2.5021  -3.8435
427  Topic12  1.596101e+04       obtain  1.596101e+04   2.5021  -3.8872
341  Topic12  1.471920e+04         cash  1.471920e+04   2.5021  -3.9682
52   Topic12  1.421492e+04     purchase  1.421492e+04   2.5021  -4.0031
425  Topic12  1.298823e+04         acct  1.298823e+04   2.5021  -4.0933
778  Topic12  1.295043e+04      account  1.295043e+04   2.5021  -4.0962
190  Topic12  1.031024e+04       number  1.031024e+04   2.5021  -4.3242
54   Topic12  9.834037e+03     identity  9.834037e+03   2.5021  -4.3715
521  Topic12  8.676456e+03      consent  8.677493e+03   2.5020  -4.4968
779  Topic12  7.414131e+03         bank  7.414131e+03   2.5021  -4.6540
56   Topic12  6.176643e+03      service  6.176643e+03   2.5021  -4.8366
799  Topic12  6.137769e+03  information  6.137769e+03   2.5021  -4.8429
416  Topic12  5.101130e+03        debit  5.1011

In [42]:
# Compute Perplexity
print('\nPerplexity: ', model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.



Perplexity:  nan


## Finding the dominant topic in each sentence

One of the practical application of topic modeling is to determine what topic a given document is about.

To find that, we find the topic number that has the highest percentage contribution in that document.

The format_topics_sentences() function below nicely aggregates this information in a presentable table.

In [43]:
def format_two_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                topic_num0 = topic_num
                prop_topic0 = prop_topic
                wp0 = ldamodel.show_topic(topic_num0)
                topic_keywords0 = ", ".join([word for word, prop in wp0])
            elif j == 1:  # => dominant topic
                topic_num1 = topic_num
                prop_topic1 = prop_topic
                wp1 = ldamodel.show_topic(topic_num1)
                topic_keywords1 = ", ".join([word for word, prop in wp1])
            else:
                break
        sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num0), round(prop_topic0,4), topic_keywords0,int(topic_num1), round(prop_topic1,4), topic_keywords1]), ignore_index=True)
    sent_topics_df.columns = ['Dominant_Topic0', 'Perc_Contribution0', 'Topic_Keywords0', 'Dominant_Topic1', 'Perc_Contribution1', 'Topic_Keywords1']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


#df_two_topic_sents_keywords = format_two_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)
df_two_topic_sents_keywords = format_two_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data)

# Format
df_two_dominant_topic = df_two_topic_sents_keywords.reset_index()
df_two_dominant_topic.columns = ['Document_No', 'Dominant_Topic0', 'Topic_Perc_Contrib0', 'Keywords0', 'Dominant_Topic1', 'Topic_Perc_Contrib1', 'Keywords1', 'Text']

# Show
df_two_dominant_topic.head(10)

KeyboardInterrupt: 

In [44]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


#df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)
df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,9.0,0.1884,"victim, suspect, state, phone, call, kill, dem...",suspect continiously calls and text victim ha...
1,1,8.0,0.1860,"victim, face, punch, suspect, time, strike, gr...",suspect approached victim and asked victim to...
2,2,1.0,0.1328,"suspect, vehicle, victim, damage, throw, drive...",suspect approached victims vehicle punctured ...
3,3,8.0,0.1427,"victim, face, punch, suspect, time, strike, gr...",unknown suspect involved in a fight inside cl...
4,4,11.0,0.1837,"suspect, item, store, pay, exit, business, ent...",suspect entered location removed and conceale...
5,5,6.0,0.1000,"suspect, victim, lock, leave, return, cut, bik...",unknown suspect cut off two cat conv from vic...
6,6,0.0,0.1269,"unknown, vehicle, window, smash, side, break, ...",unknown suspect used unknown method to enter ...
7,7,2.0,0.1580,"victim, suspect, make, card, permission, check...",unknown suspect used victims credit card info...
8,8,2.0,0.1647,"victim, suspect, make, card, permission, check...",suspect uses victims identity without permiss...
9,9,6.0,0.1192,"suspect, victim, lock, leave, return, cut, bik...",3 male asian wearings all blk female asian st...


## Find the most representative document for each topic

Sometimes just the topic keywords may not be enough to make sense of what a topic is about. So, to help with understanding the topic, you can find the documents a given topic has contributed to the most and infer the topic by reading that document. 

In [49]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.to_csv('sent_topics_sorteddf_mallet_real.csv')#.head()

In [50]:
sent_two_topics_sorteddf_mallet = pd.DataFrame()

sent_two_topics_outdf_grpd = df_two_topic_sents_keywords.groupby('Dominant_Topic0')

for i, grp in sent_two_topics_outdf_grpd:
    sent_two_topics_sorteddf_mallet = pd.concat([sent_two_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution0'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_two_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
#sent_two_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_two_topics_sorteddf_mallet.head()

NameError: name 'df_two_topic_sents_keywords' is not defined

## Topic distribution across documents

Finally, we want to understand the volume and distribution of topics in order to judge how widely it was discussed. The below table exposes that information.

In [55]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.head(12)

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,9.0,"victim, suspect, state, phone, call, kill, dem...",86980.0,0.1080
1,8.0,"victim, face, punch, suspect, time, strike, gr...",58911.0,0.0731
2,1.0,"suspect, vehicle, victim, damage, throw, drive...",92553.0,0.1149
3,8.0,"victim, face, punch, suspect, time, strike, gr...",8749.0,0.0109
4,11.0,"suspect, item, store, pay, exit, business, ent...",67629.0,0.0840
5,6.0,"suspect, victim, lock, leave, return, cut, bik...",60119.0,0.0746
6,0.0,"unknown, vehicle, window, smash, side, break, ...",60525.0,0.0751
7,2.0,"victim, suspect, make, card, permission, check...",45866.0,0.0569
8,2.0,"victim, suspect, make, card, permission, check...",88506.0,0.1099
9,6.0,"suspect, victim, lock, leave, return, cut, bik...",83534.0,0.1037


In [56]:
df_dominant_topics.head(12).to_csv('df_dominant_topics_real.csv')

In [57]:
df_dominant_topic.to_csv('df_dominant_topic_real.csv')

In [60]:
[data[0]]

[' suspect continiously calls and text victim harrassing messages victim has repeatedly advised suspect to stop making contact ']

## Sentence coherence score 

In [243]:
def check(word, list):
    if word in list:
        return 1
    else:
        return 0

In [244]:
def format_topics_sentence(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    total_words = []
    words = []
    words_topics_dics = []
    index = 0
    for cp in corpus:
        total_words.append(0)
        words.append([])
        for id, freq in cp:
            total_words[index]+=freq
            words[index].append(id2word[id])
        values = [0] * len(cp)
        words_topics_dics.append(dict(zip(words[index],values)))
        index +=1
    
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        #words[i] is list of words of the sentence 
        #need to update words_topics_dics
        words_topics_dic = words_topics_dics[i] 
        
        #row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if(j<12):
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = [word for word, prop in wp]# = ", ".join([word for word, prop in wp]) 
                for key, value in words_topics_dic.items():
                    if key in topic_keywords:
                        words_topics_dic[key]+=1
                #print(prop_topic)
                #print(topic_num)
                #print(wp)
                #print(topic_keywords)
            else:
                break
    return words_topics_dics
sentences = [data[0], data[1], data[2], data[3], data[4]]
sentence_corpus = [corpus[0], corpus[1], corpus[2], corpus[3], corpus[4]]
words_topics_dictionaries = format_topics_sentence(ldamodel=ldamallet, corpus=sentence_corpus, texts=sentences)

In [245]:
def score_calculator(wt_dics):
    scores = []
    index = 0
    for wt_dic in wt_dics:
        total_words = len(wt_dic)
        #print(total_words)
        commons_topics = dict((k, v) for k, v in wt_dic.items() if v > 1)
        unique_topics = dict((k, v) for k, v in wt_dic.items() if v <= 1)
        score = len(unique_topics) - len(commons_topics)
        scores.append(score/total_words)
    return scores
s =  score_calculator(words_topics_dictionaries)        

In [246]:
sentences

[' suspect continiously calls and text victim harrassing messages victim has repeatedly advised suspect to stop making contact ',
 ' suspect approached victim and asked victim to fight suspect hit victim multiple times victim defendedhimself and punched suspect suspect fled location southbound on kingsley from melrose ',
 ' suspect approached victims vehicle punctured victims tires with an unknown sharp tool and fled location in unknown dir ',
 ' unknown suspect involved in a fight inside club hit victim in back of head ',
 ' suspect entered location removed and concealed selected items suspect then fled locvia exit without paying for items ']

In [247]:
words_topics_dictionaries

[{'advise': 0,
  'call': 1,
  'contact': 0,
  'continiously': 0,
  'harrasse': 0,
  'make': 1,
  'message': 0,
  'repeatedly': 0,
  'stop': 0,
  'suspect': 8,
  'text': 0,
  'victim': 7},
 {'suspect': 8,
  'victim': 7,
  'approached': 1,
  'ask': 0,
  'fight': 0,
  'flee': 4,
  'hit': 1,
  'location': 3,
  'multiple': 0,
  'punch': 1,
  'time': 1},
 {'suspect': 8,
  'victim': 7,
  'flee': 4,
  'location': 3,
  'approach': 1,
  'dir': 1,
  'puncture': 0,
  'sharp': 0,
  'tire': 0,
  'tool': 1,
  'unknown': 5,
  'vehicle': 1},
 {'suspect': 8,
  'victim': 7,
  'fight': 0,
  'hit': 1,
  'unknown': 5,
  'club': 0,
  'head': 1,
  'involve': 0},
 {'suspect': 8,
  'flee': 4,
  'location': 3,
  'conceal': 0,
  'entered': 1,
  'exit': 1,
  'item': 1,
  'locvia': 0,
  'pay': 1,
  'remove': 3,
  'select': 0}]

In [248]:
s

[0.6666666666666666,
 0.2727272727272727,
 0.16666666666666666,
 0.25,
 0.2727272727272727]

In [222]:
sentences = [data[0]]
sentence_corpus = [corpus[0]]

In [223]:
sentences

[' suspect continiously calls and text victim harrassing messages victim has repeatedly advised suspect to stop making contact ']

In [224]:
sentence_corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 2),
  (10, 1),
  (11, 2)]]

((number of words that are present in only one topic) - (number of words that are from multi topics))/(total number of words in the document)

In [107]:
corpus[0:1]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 2),
  (10, 1),
  (11, 2)]]

In [106]:
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 2),
 (10, 1),
 (11, 2)]

In [101]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[0:1]]

[[('advise', 1),
  ('call', 1),
  ('contact', 1),
  ('continiously', 1),
  ('harrasse', 1),
  ('make', 1),
  ('message', 1),
  ('repeatedly', 1),
  ('stop', 1),
  ('suspect', 2),
  ('text', 1),
  ('victim', 2)]]

In [249]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("ldamallet")
ldamallet.save(temp_file)


In [250]:
# Load a potentially pretrained model from disk.
saved_model = gensim.models.ldamodel.LdaModel.load(temp_file)

In [181]:
texts

[['suspect',
  'continiously',
  'call',
  'text',
  'victim',
  'harrasse',
  'message',
  'victim',
  'repeatedly',
  'advise',
  'suspect',
  'stop',
  'make',
  'contact'],
 ['suspect',
  'approached',
  'victim',
  'ask',
  'victim',
  'fight',
  'suspect',
  'hit',
  'victim',
  'multiple',
  'time',
  'victim',
  'punch',
  'suspect',
  'suspect',
  'flee',
  'location'],
 ['suspect',
  'approach',
  'victim',
  'vehicle',
  'puncture',
  'victim',
  'tire',
  'unknown',
  'sharp',
  'tool',
  'flee',
  'location',
  'unknown',
  'dir'],
 ['unknown', 'suspect', 'involve', 'fight', 'club', 'hit', 'victim', 'head'],
 ['entered',
  'location',
  'remove',
  'conceal',
  'select',
  'item',
  'suspect',
  'flee',
  'locvia',
  'exit',
  'pay',
  'item'],
 ['unknown', 'suspect', 'cut', 'victim', 'vehicle'],
 ['unknown',
  'suspect',
  'use',
  'unknown',
  'method',
  'enter',
  'victim',
  'vehicle',
  'attempt',
  'punch',
  'vehicle',
  'ignition',
  'dash',
  'victim',
  'propert

In [173]:
# Create a new corpus, made of previously unseen documents.

g_df = pd.read_csv('./generated/seqgan_new.csv', quoting = 3).dropna().reset_index(drop=True)
g_df['content'] = " " + g_df['NARRATIVE'].str.lower()+ " "

for key, value in word_replace_dic.items():
    g_df['content'] = g_df['content'].str.replace(key, value, 10)

In [174]:
# Convert to list
g_data = g_df.content.values.tolist()

# Remove Emails
g_data = [re.sub('\S*@\S*\s?', '', sent) for sent in g_data]
# Remove new line characters
g_data = [re.sub('\s+', ' ', sent) for sent in g_data]
# Remove distracting single quotes
g_data = [re.sub("\'", "", sent) for sent in g_data]
g_data_words = list(sent_to_words(g_data))

# Remove Stop Words
g_data_words_nostops = remove_stopwords(g_data_words)
# Form Bigrams
g_data_words_bigrams = make_bigrams(g_data_words_nostops)
# Do lemmatization keeping only noun, adj, vb, adv
g_data_lemmatized = lemmatization(g_data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(g_data_lemmatized[:1])

# Create Corpus
#texts = data_lemmatized

# Create Dictionary
#id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
#texts = data_lemmatized

# Term Document Frequency
#corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

other_texts = g_data_lemmatized
other_corpus = [id2word.doc2bow(text) for text in other_texts]

[['unknown', 'suspect', 'email', 'victim', 'state', 'rpt', 'victim', 'discover', 'life', 'letter']]


In [176]:
g_sentences = [g_data[0], g_data[1], g_data[2], g_data[3], g_data[4]]
unseen_doc = [other_corpus[0], other_corpus[1], other_corpus[2], other_corpus[3], other_corpus[4]]
#vector = saved_model[unseen_doc]  # get topic probability distribution for a document
g_words_topics_dictionaries = format_topics_sentence(ldamodel=ldamallet, corpus=unseen_doc, texts=sentences)

In [177]:
s =  score_calculator(g_words_topics_dictionaries)        

In [178]:
g_words_topics_dictionaries

[{'suspect': 9,
  'victim': 7,
  'unknown': 4,
  'state': 1,
  'discover': 1,
  'life': 0,
  'letter': 0,
  'email': 0,
  'rpt': 0},
 {'suspect': 9,
  'victim': 7,
  'unknown': 4,
  'vehicle': 2,
  'remove': 3,
  'property': 3,
  'smash': 1,
  'window': 1,
  'car': 1,
  'fear': 1,
  'poe': 0,
  'give': 0,
  'beer': 0},
 {'suspect': 9, 'victim': 7, 'state': 1, 'shoot': 0, 'build': 0, 'bitch': 0},
 {'suspect': 9,
  'victim': 7,
  'unknown': 4,
  'vehicle': 2,
  'passenger': 0,
  'roll': 0,
  'door': 1,
  'cause': 0,
  'damage': 1,
  'rent': 0,
  'dent': 0,
  'rmv': 0,
  'deep': 0},
 {'suspect': 9,
  'victim': 7,
  'time': 1,
  'property': 3,
  'strike': 1,
  'become': 0,
  'dispute': 1,
  'enrage': 0,
  'work': 0,
  'several': 0,
  'hair': 0,
  'believe': 0,
  'street': 0}]

In [179]:
s

[0.3333333333333333,
 0.07692307692307693,
 0.3333333333333333,
 0.38461538461538464,
 0.5384615384615384]

In [180]:
g_sentences

[' unknown suspect pry hrd obj email victim state rpt victim discover life doesnt letter ',
 ' unknown suspects pry obj pry vehicle smash car window vehicle remove property poe victim fear give anyone property smash car beer ',
 ' victim trespass suspect state bitch build shoot ',
 ' unknown suspect rmvs rent vehicle roll passenger door damage victim vehicle around vehicle cause deep dent ',
 ' suspect become enrage victim dispute victim property dispute victim hair victim believe suspect work street strike victim several time ']

## Generate coherence score for table reported in the paper


In [251]:
# Create a new corpus, made of previously unseen documents.

t_df = pd.read_excel('./generated/table_content.xlsx')#, error_bad_lines=False,names=('NARRATIVE'))

t_df.columns

Index(['NARRATIVE'], dtype='object')

In [252]:

t_df['content'] = " " + t_df['NARRATIVE'].str.lower()+ " "

for key, value in word_replace_dic.items():
    t_df['content'] = t_df['content'].str.replace(key, value, 10)

In [253]:
# Convert to list
t_data = t_df.content.values.tolist()

# Remove Emails if available
t_data = [re.sub('\S*@\S*\s?', '', sent) for sent in t_data]
# Remove new line characters
t_data = [re.sub('\s+', ' ', sent) for sent in t_data]
# Remove distracting single quotes
t_data = [re.sub("\'", "", sent) for sent in t_data]
t_data_words = list(sent_to_words(t_data))

# Remove Stop Words
t_data_words_nostops = remove_stopwords(t_data_words)
# Form Bigrams
t_data_words_bigrams = make_bigrams(t_data_words_nostops)
# Do lemmatization keeping only noun, adj, vb, adv
t_data_lemmatized = lemmatization(t_data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(t_data_lemmatized[:1])

other_texts = t_data_lemmatized
other_corpus = [id2word.doc2bow(text) for text in other_texts]

[['suspect', 'continiously', 'call', 'text', 'victim', 'harrasse', 'message', 'victim', 'repeatedly', 'advise', 'suspect', 'stop', 'make', 'contact']]


In [258]:
t_sentences = t_data[:45]
unseen_doc = other_corpus[:45]

In [259]:
t_sentences

[' suspect continiously calls and text victim harrassing messages victim has repeatedly advised suspect to stop making contact ',
 ' suspect approached victim and asked victim to fight suspect hit victim multiple times victim defendedhimself and punched suspect suspect fled location southbound on kingsley from melrose ',
 ' suspect approached victims vehicle punctured victims tires with an unknown sharp tool and fled location in unknown dir ',
 ' unknown suspect involved in a fight inside club hit victim in back of head ',
 ' suspect entered location removed and concealed selected items suspect then fled locvia exit without paying for items ',
 ' a suspect entered res thru unlocked door removed victim rings rack when victim checked but fled location in unknown dir ',
 ' unknown suspect approached victim vehicle broke glass window to victim bedroom causing damage to bend ',
 ' suspects spray painted graffiti on victims balcony window hitting front walls ',
 ' suspect approached victim a

In [260]:
#vector = saved_model[unseen_doc]  # get topic probability distribution for a document
t_words_topics_dictionaries = format_topics_sentence(ldamodel=ldamallet, corpus=unseen_doc, texts=t_sentences)

In [262]:
s =  score_calculator(t_words_topics_dictionaries)        
pprint(s)

[0.6666666666666666,
 0.2727272727272727,
 0.16666666666666666,
 0.25,
 0.2727272727272727,
 0.07692307692307693,
 0.5,
 0.42857142857142855,
 0.5555555555555556,
 0.5714285714285714,
 -0.25,
 0.4,
 0.2,
 0.5,
 0.5,
 0.6666666666666666,
 0.625,
 0.5555555555555556,
 0.6,
 0.8571428571428571,
 0.8,
 0.3333333333333333,
 0.2,
 1.0,
 1.0,
 0.09090909090909091,
 0.125,
 0.0,
 0.29411764705882354,
 0.3333333333333333,
 0.3333333333333333,
 0.0,
 0.7142857142857143,
 0.38461538461538464,
 -0.7142857142857143,
 0.0,
 0.3333333333333333,
 0.0,
 -0.5,
 0.7391304347826086,
 0.6,
 0.625,
 0.5,
 0.0,
 0.5]
